# Welcome!

In this initial part we import all the packages we might need (a lot of them!)

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
import math
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
from geometry_msgs.msg import Twist
from rt2_assignment1.srv import Command
from tf import transformations
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox

rospy.init_node('robot_control_node')

## Input

### Buttons to start and stop robot random behavior

We initialize a user interface client to send our commands.

In [2]:
ui_client = rospy.ServiceProxy('/user_interface', Command)

Here we define three beautiful buttons for starting random behavior, stopping it and stopping it immediately.

In [3]:
start = Button(description='START',
layout=Layout(width='10%', align="center", grid_area='start'),
style=ButtonStyle(button_color='lightgreen'))

stop = Button(description='STOP',
layout=Layout(width='10%', align="center", grid_area='stop'),
style=ButtonStyle(button_color='moccasin'))

stop_now = Button(description='STOP NOW',
layout=Layout(width='10%', align="center", grid_area='stop_now'),
style=ButtonStyle(button_color='salmon'))

HBox([start, stop, stop_now])

And finally we define their behavior upon being clicked. Each button sends its own request to the *user_interface* server, corresponding to the requests in the original assignment solution.

In [4]:
def on_start_clicked(b):
    ui_client("start")
    
def on_stop_clicked(b):
    ui_client("stop")
    
def on_stopnow_clicked(b):
    ui_client("stop_now")
    
start.on_click(on_start_clicked)
stop.on_click(on_stop_clicked)
stop_now.on_click(on_stopnow_clicked)

### Sliders to change velocity

Sliders are directly linked to **ROS Parameter Server**. Every time a slider value is changed, the corresponding parameter is being set to the new value.

In [5]:
lin_vel = widgets.FloatSlider(min=0.0, max=2.0, value=1.0, description="Linear")
display(lin_vel)

ang_vel = widgets.FloatSlider(min=0.0, max=2.0, value=1.0, description="Angular")
display(ang_vel)

def on_valuelin_change(change):
    rospy.set_param('lin_vel', change['new'])
    
def on_valueang_change(change):
    rospy.set_param('ang_vel', change['new'])

lin_vel.observe(on_valuelin_change, names='value')
ang_vel.observe(on_valueang_change, names='value')

FloatSlider(value=1.0, description='Linear', max=2.0)

FloatSlider(value=1.0, description='Angular', max=2.0)

### Manual control buttons

In [6]:
forward = Button(description='\u2191',
layout=Layout(width='auto', align="center", grid_area='forward'),
style=ButtonStyle(button_color='lightgreen'))

backward = Button(description='\u2193',
layout=Layout(width='auto', align="center", grid_area='backward'),
style=ButtonStyle(button_color='lightgreen'))

left = Button(description='\u21b6',
layout=Layout(width='auto', align="center", grid_area='left'),
style=ButtonStyle(button_color='lightgreen'))

right = Button(description='\u21b7',
layout=Layout(width='auto', align="center", grid_area='right'),
style=ButtonStyle(button_color='lightgreen'))

cancel = Button(description='O',
layout=Layout(width='auto', align="center", grid_area='cancel'),
style=ButtonStyle(button_color='lightgreen'))

GridBox(children=[forward, backward, left, right, cancel],
layout=Layout(
width='10%',
grid_template_rows='33% 33% 33%',
grid_template_columns='33% 33% 33%',
grid_template_areas='''
" . forward . "
"left cancel right "
" . backward . "
''')
)

GridBox(children=(Button(description='↑', layout=Layout(grid_area='forward', width='auto'), style=ButtonStyle(…

In [7]:
pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)

def on_forward_clicked(b):
    msg = Twist()
    msg.linear.x = rospy.get_param('lin_vel')
    pub.publish(msg)
    
def on_backward_clicked(b):
    msg = Twist()
    msg.linear.x = -rospy.get_param('lin_vel')
    pub.publish(msg)
    
def on_left_clicked(b):
    msg = Twist()
    msg.angular.z = -rospy.get_param('ang_vel')
    pub.publish(msg)
    
def on_right_clicked(b):
    msg = Twist()
    msg.angular.z = rospy.get_param('ang_vel')
    pub.publish(msg)
    
def on_cancel_clicked(b):
    msg = Twist()
    pub.publish(msg)
    
forward.on_click(on_forward_clicked)
backward.on_click(on_backward_clicked)
left.on_click(on_left_clicked)
right.on_click(on_right_clicked)
cancel.on_click(on_cancel_clicked)

## Output

### A position plot

In [8]:
fig, ax = plt.subplots()
fig.suptitle('Robot position and orientation plot', fontsize=16)

ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))
ax.set_aspect("equal")

line, = ax.plot([], [], 'ro')
an1 = ax.annotate("", xy=(0, 1), xytext=(0, 0), arrowprops={"facecolor": "black"})
#orient, = ax.plot([], [], 'ko')
x_data=[]
y_data=[]
yaw = 0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
def init():
    line.set_data([], [])
    #orient.set_data([], [])
    return (line, )


def odom_callback(msg):
    global yaw
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]


def animate(i):
    global yaw
    line.set_data(x_data, y_data)
    #endy = y_data[-1] + 1 * math.sin(math.radians(angle))
    #endx = x_data[-1] + 1 * math.cos(math.radians(angle))
    an1.xy = (x_data[-1] + math.cos(yaw), y_data[-1] + math.sin(yaw))
    an1.set_position((x_data[-1], y_data[-1]))
    #orient.set_data(endx, endy)
    #ax.arrow(x_data[-1],y_data[-1],math.cos(yaw),math.sin(yaw))
    return (line, an1)



jr.subscribe('/odom', Odometry, odom_callback)

In [10]:
# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)